In [ ]:
import numpy as np
import nibabel as nib
import os,glob,cv2,re,json,numbers
import pylab as plt
import pandas as pd
from pathlib import *
from scipy import ndimage
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.layers import BatchNormalization, Dropout, Activation
from tensorflow.keras.layers import Dense, Flatten, Input
from keras.layers.merge import add
from keras.regularizers import l2
kernel_regularizer=l2(1e-4)

In [ ]:
def read_nifti_file(filepath):
    scan = nib.load(filepath)
    scan = scan.get_fdata()
    return scan


def normalize(volume):
    min = -200
    max = 1200
    #float(volume.min())= min
    #float(volume.max()) = max
    #volume = (volume - min) / (max - min)
    #volume = volume.astype("float32")
    volume = np.floor((volume - min) / (max - min))
    return volume


def resize_volume(img):
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    img = ndimage.rotate(img, 90, reshape=False)
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img


def process_scan(path):
    volume = read_nifti_file(path)
    volume = normalize(volume)
    volume = resize_volume(volume)
    return volume

In [ ]:
glob_path_control = glob.glob(r"../input/parkinson/taowu/sub-control*/anat/*.nii")

glob_path_patient = glob.glob(r"../input/parkinson/taowu/sub-patient*/anat/*.nii")

control_data = np.array([process_scan(path) for path in glob_path_control])
patient_data = np.array([process_scan(path) for path in glob_path_patient])


control_labels = np.array([1 for _ in range(len(control_data))])
patient_labels = np.array([0 for _ in range(len(patient_data))])


X = np.concatenate((control_data, patient_data), axis=0)
print("Dataset Shape : ", X.shape)
Y = np.concatenate((control_labels, patient_labels), axis=0)
print("Label Shape : ", Y.shape)

In [ ]:
Y = to_categorical(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)
print("X_train : ", X_train.shape)
print("X_test : ", X_test.shape)
print("Y_train : ", Y_train.shape)
print("Y_test : ", Y_test.shape)

In [ ]:
import random
from scipy import ndimage
@tf.function
def rotate(volume):
    """Rotate the volume by a few degrees"""

    def scipy_rotate(volume):
        # define some rotation angles
        angles = [-20, -10, -5, 5, 10, 20]
        # pick angles at random
        angle = random.choice(angles)
        # rotate volume
        volume = ndimage.rotate(volume, angle, reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume


def train_preprocessing(volume, label):
    """Process training data by rotating and adding a channel."""
    # Rotate volume
    volume = rotate(volume)
    volume = tf.expand_dims(volume, axis=3)
    return volume, label


def validation_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label

In [ ]:
train_loader = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
validation_loader = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

batch_size = 2
# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(X_train))
    .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)
# Only rescale.
validation_dataset = (
    validation_loader.shuffle(len(X_test))
    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

In [ ]:
nb_class = 2
opt = 'adam'
classifier = 'sigmoid'
loss_function = 'binary_crossentropy'


def get_model(width=128, height=128, depth=64, channel = 1):

    inputs = Input((width, height, depth, channel))
    
 
    conv1 = Conv3D(filters=32, kernel_size=(5, 5, 5),
                           strides=(2,2,2), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(inputs)
    print(conv1.shape)
    conv11 = Conv3D(filters=32, kernel_size=(5, 5, 5),
                           strides=(1,1,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(conv1)
    norm1 = BatchNormalization(axis=-1)(conv11)
    relu1 = Activation("relu")(norm1)
    print(relu1.shape)
    residual1 = Conv3D(filters=32, kernel_size=(3, 3, 3),
                           strides=(1,1,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(relu1)
    print(residual1.shape)
    resblock1 = add([conv1, residual1])
    
    conv2 = Conv3D(filters=64, kernel_size=(5, 5, 5),
                           strides=(2,2,2), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(resblock1)
    
    conv22 = Conv3D(filters=64, kernel_size=(5, 5, 5),
                           strides=(1,1,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(conv2)
    norm2 = BatchNormalization(axis=-1)(conv22)
    relu2 = Activation("relu")(norm2)
    print(relu1.shape)
    residual2 = Conv3D(filters=64, kernel_size=(3, 3, 3),
                           strides=(1,1,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(relu2)
    print(residual1.shape)
    resblock2 = add([conv2, residual2])
    
    
    conv3 = Conv3D(filters=64, kernel_size=(3, 3, 3),
                           strides=(2,2,2), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(resblock2)
    
    conv33 = Conv3D(filters=128, kernel_size=(3, 3, 3),
                           strides=(1,1,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(conv3)
    norm3 = BatchNormalization(axis=-1)(conv3)
    relu3 = Activation("relu")(norm3)
    print(relu1.shape)
    residual3 = Conv3D(filters=64, kernel_size=(3, 3, 3),
                           strides=(1,1,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(relu3)
    print(residual1.shape)
    resblock3 = add([conv3, residual3])
    
    conv4 = Conv3D(filters=16, kernel_size=(3, 3, 3),
                           strides=(2,2,2), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(resblock3)
    
    conv44 = Conv3D(filters=16, kernel_size=(3, 3, 3),
                           strides=(1,1,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(conv4)
    norm4 = BatchNormalization(axis=-1)(conv44)
    relu4 = Activation("relu")(norm4)
    print(relu1.shape)
    residual4 = Conv3D(filters=16, kernel_size=(3, 3, 3),
                           strides=(1,1,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(relu4)
    print(residual1.shape)
    resblock4 = add([conv4, residual4])
    
    conv5 = Conv3D(filters=16, kernel_size=(3, 3, 3),
                           strides=(2,2,1), padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=kernel_regularizer
                           )(resblock4)
    
    x = MaxPooling3D(pool_size=(2,2,2), strides=(2, 2, 2))(conv5)
    y = Flatten()(x)
    outputs = Dense(units=nb_class, activation=classifier,
                    kernel_initializer ='he_normal')(y)
    
  
    model = Model(inputs, outputs, name="Resnet")
    
    return model


model = get_model(width=128, height=128, depth=64, channel =1)

#print(model.summary())

In [ ]:
# Compile model.
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc"],
)

# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image_classification.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

# Train the model, doing validation at the end of each epoch
epochs = 100
model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs,
    shuffle=True,
    verbose=2,
    callbacks=[checkpoint_cb, early_stopping_cb],
)


In [ ]:
model.compile(optimizer = opt,
                loss = loss_function,
                metrics = ['accuracy'])
history = model.fit(train_dataset, validation_dataset, 
                         batch_size = 64, 
                         verbose = 1, 
                         epochs = 90,      
                         validation_data=(X_test,Y_test),
                         shuffle = True)

In [ ]:
import matplotlib.pyplot as plt
loss_and_metrics = model.evaluate(x_test, y_test, verbose=1)
print("Test Loss", loss_and_metrics[0])
print("Test Accuracy", loss_and_metrics[1])

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')

In [ ]:
i